# How would ReproNim do that?

## Volume 4: Manage local FreeSurfer results

This is the companion Jupyter notebook to *How would ReproNim do that? Volume 4*.

### Problem statement

I have performed a FreeSurfer analysis locally. I hear that ReproNim has some cool tools to manage the results of such an analysis. What are they and how do I do it?

In this example we have run FreeSurfer 6.0 on a set of 17 ABIDE subjects. The FreeSurfer SUBJECTS_DIR is subjects/ and is stripped down to contain only stats/aseg.stats, stats/lh.aparc.stats, stats/rh.aparc.stats, and scripts/build-stamp.txt for each subject.

### In theory

Given the FreeSurfer results, we will use segstats_jsonld to create a NeuroImaging Data Model (NIDM) semantically marked up version of the volumetric data files. The semantic markup makes the FreeSurfer results self-describing in that each result is associated with a concept that completely describes the result (type of measure (e.g. volume), structure (e.g. caudate nucleus), etc.). Complete semantic description of results makes integration of results from disparate sources easier. Once we’ve created the NIDM version of the FreeSurfer results, we can query our results using PyNIDM, and, more importantly, we can merge local results with publicly available results and query across these combined data sets.

See the static document for instructions on installing segstats_jsonld, which provides the programs used below.

The combined time for running all the cells in this notebook is approximately ten minutes.

We start by running asegstats2nidm on a single subject in order to express local FreeSurfer results in NIDM.

In [ ]:
mkdir single_subject

In [ ]:
segstats2nidm -add_de -s `pwd`/subjects/Olin_0050107 -o `pwd`/single_subject

This takes the aseg.stats file for my particular subject (Olin_0050107) and generates the following files:

In [ ]:
ls single_subject

These files contain NIDM data in Turtle format (.ttl).

Examine a NIDM file to see what it has using `pynidm query`.

*pynidm is installed when segstats_jsonld is installed.  `-nl` is short for `--nidm_file_list` and specifies the input files.  `-bv` is short for `--get_brainvols` and instructs pynidm to run a query for brain volumes.  `` `pwd` `` is needed because pynidm requires absolute paths.*

In [ ]:
pynidm query -nl `pwd`/single_subject/aseg_nidm.ttl -bv -o `pwd`/single_query_results.txt

In [ ]:
cat `pwd`/single_query_results.txt

So that was one subject, but we have a set of 17 FreeSurfer subjects.  Let’s make a NIDM representation of this complete set.

In [ ]:
mkdir all_subjects

*This step will take about five minutes to run.*

In [ ]:
for subject_dir in subjects/*
do
    subject=`basename $subject_dir`
    echo $subject
    mkdir all_subjects/$subject
    segstats2nidm -add_de -s `pwd`/$subject_dir -o `pwd`/all_subjects/$subject
    echo
done
echo done

This results in in a directory (all_subjects) with subdirectories for each subject (CMU_0050646, Caltech_51478, etc.) which contain the corresponding aseg_nidm.ttl, lh.aparc_nidm.ttl, and rh.aparc_nidm.ttl files.

It is unwieldy to work with many separate sets of files, so we can combine the relevant .ttl files into one. For the examples that follow, we will only need the segmentation results, so we concatenate the aseg NIDM files from each of the subjects into one file.

*pynidm requires the input files to be specified as a comma-separated list, so we create that first.  `pynidm concat` will take about a minute to run.*

In [ ]:
aseg_list=`echo all_subjects/*/aseg_nidm.ttl | tr ' ' ,`

In [ ]:
pynidm concat -nl $aseg_list -o all_subjects/all_aseg.ttl

We can now query this combined file for various content (remember that all this file has is the regional volumes per subject). First, let’s just look at the whole file for brain volumes (-bv):

In [ ]:
pynidm query -nl all_subjects/all_aseg.ttl -bv -o all_query_results.txt

In [ ]:
cat all_query_results.txt

Next, we can ask for a specific brain structure (i.e. hippocampus) by providing the appropriate SPARQL query.

*We provide a prepared query for finding hippocampus volumes.  Other sample queries can be found at https://github.com/dbkeator/simple2_NIDM_examples/tree/master/queries.*

In [ ]:
pynidm query -nl all_subjects/all_aseg.ttl -q hippocampus_query.sparql -o all_hippocampus_results.txt

In [ ]:
cat all_hippocampus_results.txt

This is my local NIDM "pool" of FreeSurfer data from this particular study.

It would be more useful to include demographics and other clinical/behavioral variables, so we add demographics.

For this example, demographics (age (in years), sex, and group) are in a CSV file (demographics.csv). We want to express this data in the NIDM file containing our FreeSurfer data (demographics.ttl) so that it can be queried and used for analysis.  There are two options: use csv2nidm and choose to add the demographics data to an existing NIDM file, or run csv2nidm without adding to an existing NIDM file and use `pynidm merge -s` later to merge NIDM files by subject ID.

In this case we do the latter.  The command to run is:

    csv2nidm -csv demographics.csv -out demographics.ttl

Because running csv2nidm is an interactive task that requires careful attention to detail and an account and API key for InterLex, this has been run in advance and demographics.ttl has already been generated.  See the static document for an example of how csv2nidm is run.

At this point you have two separate NIDM files, one with demographics data and one with FreeSurfer data.  Let’s merge both together by the subject identifiers.

In [ ]:
pynidm merge -nl demographics.ttl,all_subjects/all_aseg.ttl -s -o demog_aseg.ttl

Now demog_aseg.ttl has both your demographics data and FreeSurfer brain volumes.

Now that we’ve merged our demographics data with our existing FreeSurfer NIDM file, we can query this combined file get the hippocampus volumes and subject ages.

First we need to query for the project identifier in the NIDM file.

*`-u` instructs pynidm to run a query as a REST API URI.*

In [ ]:
pynidm query -nl demog_aseg.ttl -u /projects

Next let’s find out what we have available for hippocampus statistics.

In [ ]:
pynidm query -nl demog_aseg.ttl -u /projects/a3814f98-8993-11ea-9534-f01898563600 | grep Hippo

Of the results above, we’re interested in the left hippocampus volume, which has ID "fs_003343."  Next let’s see if we have age in the dataset.

In [ ]:
pynidm query -nl demog_aseg.ttl -u /projects/a3814f98-8993-11ea-9534-f01898563600 | grep age

It looks like we have a variable for age.  Now let’s query the NIDM file and ask for only these two fields of interest (notice the `?fields=` part of the query) and save them to an output file for further analysis.

In [ ]:
pynidm query -nl demog_aseg.ttl -u /projects/a3814f98-8993-11ea-9534-f01898563600?fields=age,fs_003343 -o age_hippo_raw.csv

In [ ]:
cat age_hippo_raw.csv

Note that volumes and ages are on separate lines.  We want to join them so each subject’s age and left hippocampus volume are on the same line.

*`tail -n +2` skips the header line.  The paste command then joins pairs of lines.*

In [ ]:
tail -n +2 age_hippo_raw.csv  | paste -d , - - > age_hippo.csv

In [ ]:
cat age_hippo.csv

Each line contains comma-separated fields, the sixth of which is the left hippocampus volume, and the twelfth of which is the age.

To plot the result, we use a Python script that we will reuse later.

In [ ]:
cat plot

We plot the data:

In [ ]:
./plot age_hippo.csv vol_vs_age.png

In [ ]:
display < vol_vs_age.png

Now we add public data. Let’s grab the publicly accessible version of the FreeSurfer analysis of the ADHD-200 data set:

In [ ]:
curl -o adhd200_brown.ttl https://raw.githubusercontent.com/dbkeator/simple2_NIDM_examples/master/datasets.datalad.org/adhd200/RawDataBIDS/Brown/nidm.ttl

Now we query the ADHD-200 data as we did our local data above.  The concept for the left hippocampus volume is the same as above, so we use `fields=age,fs_003343` again and just update the project ID.

In [ ]:
pynidm query -nl adhd200_brown.ttl -u /projects

*This command will take approximately one minute.*

In [ ]:
pynidm query -nl adhd200_brown.ttl -u /projects/aedd7d40-6181-11ea-ab38-003ee1ce9545?fields=age,fs_003343 -o adhd200_age_hippo_raw.csv

And we again join the age and volume lines.

In [ ]:
tail -n +2 adhd200_age_hippo_raw.csv  | paste -d , - - > adhd200_age_hippo.csv

Now we use our plotting script to plot the local and ADHD-200 data together.

In [ ]:
./plot age_hippo.csv adhd200_age_hippo.csv all_vol_vs_age.png

In [ ]:
display < all_vol_vs_age.png